In [ ]:
# Version History
#print('Version 1.0.0: 09/13/2022 03:31pm - Nate Calvanese - Initial Version')
#print('Version 1.0.1: 09/14/2022 09:04pm - Nate Calvanese - Additional functionality to cover remainder of CMG')
#print('Version 1.0.2: 09/16/2022 11:37am - Nate Calvanese - Fixed bug in multi-column array agg when source column was an array itself')
#print('Version 1.0.3: 09/19/2022 9:40am - Nate Calvanese - Added in vocab map evaluation query construction')
#print('Version 1.0.4: 09/20/2022 4:29pm - Nate Calvanese - Changed record set validation logic')
#print('Version 1.0.5: 09/21/2022 11:58am - Nate Calvanese - Made multi-column array agg return array with distinct values')
#print('Version 1.0.6: 10/6/2022 9:17pm - Nate Calvanese - Added support for multiple queries for the same table')
#print('Version 1.0.7: 10/15/2022 3:08pm - Nate Calvanese - Added support for column aggregation logic')
print('Version 1.0.8: 10/24/2022 4:24pm - Nate Calvanese - Added UUID validation transform function')


In [ ]:
#!pip install data_repo_client

In [ ]:
## Imports and environment variables

# Imports
import json
import re
import os
from itertools import combinations
from google.cloud import bigquery
import pandas as pd

# Workspace environment variables
ws_name = os.environ["WORKSPACE_NAME"]
ws_project = os.environ["WORKSPACE_NAMESPACE"]
ws_bucket = os.environ["WORKSPACE_BUCKET"]
ws_bucket_name = re.sub('^gs://', '', ws_bucket)

# print(f"workspace name = {ws_name}")
# print(f"workspace project = {ws_project}")
# print(f"workspace bucket = {ws_bucket}")
# print(f"workspace bucket name = {ws_bucket_name}")


In [ ]:
## Support Functions

# Function to validate the syntax of a BQ SQL query
def run_syntax_check(query):
    try:
        client = bigquery.Client()
        client.query(query).result()
        return "Passed"
    except Exception as e:
        return "Failed: {}".format(e)

# Function to identify if a particular table and column is present in the source
def confirm_column_exists(schema, table, column):
    column_exists = False
    for table_entry in schema["tables"]:
        if table_entry["name"] == table:
            for column_entry in table_entry["columns"]:
                if column_entry["name"] == column:
                    column_exists = True
    return column_exists

# Function to apply table alias and qualified name for readability and testing
def apply_table_alias(query_str, lookup_dict):
    for key in lookup_dict:
        query_str = query_str.replace(" " + key + ".", " " + lookup_dict[key]["alias"] + ".")
        query_str = query_str.replace("(" + key + ".", "(" + lookup_dict[key]["alias"] + ".")
        query_str = query_str.replace("[" + key + ".", "[" + lookup_dict[key]["alias"] + ".")
        query_str = query_str.replace("FROM " + key, "FROM " + lookup_dict[key]["qualified_name"] + " " + lookup_dict[key]["alias"])
        query_str = query_str.replace("JOIN " + key, "JOIN " + lookup_dict[key]["qualified_name"] + " " + lookup_dict[key]["alias"])
    return query_str

# Function to return table name from fully qualified field name
def split_field_name(field):
    field_split = field.split(".")
    return field_split[0], ".".join(field_split[1:])

# Function to validate whether an attribute should be processed -- Exists in target schema and at least one source field exists in the source schema
def validate_attribute(attr_dict, src_schema, tar_schema):
    valid_target = False
    valid_src = False
    # Confirm attribute exists in target schema
    for col in tar_schema["columns"]:
        if attr_dict["name"] == col["name"]:
            valid_target = True
    # Confirm source field exists in source schema (single source field)
    if len(attr_dict["source"]["fields"]) == 1:
        tab_name, col_name = split_field_name(attr_dict["source"]["fields"][0])
        valid_src = confirm_column_exists(src_schema, tab_name, col_name)
        if valid_target and valid_src:
            return True
        else:
            return False
    # Confirm source fields exist in source schema, if marked as required or used in condition/transform/aggregation (multiple source fields)
    elif len(attr_dict["source"]["fields"]) > 1:
        if attr_dict["source"]["all_fields_required"] == True:
            found_counter = 0
            for fld in attr_dict["source"]["fields"]:
                tab_name, col_name = split_field_name(fld)
                valid_src = confirm_column_exists(src_schema, tab_name, col_name)
                if not valid_src:
                    return False
                else:
                    found_counter += 1
            if valid_target and found_counter == len(attr_dict["source"]["fields"]):
                return True
        else:
            field_counter = 0
            for fld in attr_dict["source"]["fields"]:
                tab_name, col_name = split_field_name(fld)
                valid_src = confirm_column_exists(src_schema, tab_name, col_name)
                if not valid_src:
                    if attr_dict["source"]["under_condition"] != None:
                        if fld in attr_dict["source"]["under_condition"]:
                            return False
                    for transform in attr_dict["source"]["with_transformation"]:
                        for parameter in transform["parameters"]:
                            if fld in parameter:
                                return False
                    if attr_dict["source"]["with_row_aggregation"] != None:
                        if fld in attr_dict["source"]["with_row_aggregation"]["partition_by"] or fld in attr_dict["source"]["with_row_aggregation"]["order_by"]:
                            return False
                else:
                    field_counter += 1
            if valid_target and field_counter > 0:
                return True
    else:
        if valid_target:
            return True

# Function to validate whether a query should be built for a record set -- Primary key attributes are all found and have valid specifications
def validate_record_set(map_set, src_schema_dict, target_table):
    target_pk_cols = target_table["primaryKey"]
    map_attr_list = [attr["name"] for attr in map_set["attributes"]]
    # Confirm all pk_cols are present and have valid specifications for the record set
    for pk in target_pk_cols:
        if pk not in map_attr_list:
            return False
        else:
            for attr in map_set["attributes"]:
                if attr["name"] == pk and not validate_attribute(attr, src_schema_dict, target_table):
                    return False
    # Confirm any other columns marked as required have valid specifications for the record set
    for attr in map_set["attributes"]:
        if attr["required"] and not validate_attribute(attr, src_schema_dict, target_table):
            return False
    return True
        
# Function to find and return a direct relationship between two tables, if one exists
def confirm_relationship(rel_list, tab1, tab2):
    relationship_exists = False
    relationship = {}
    for rel in rel_list:
        if (rel["_from"]["table"] == tab1 and rel["to"]["table"] == tab2) or (rel["_from"]["table"] == tab2 and rel["to"]["table"] == tab1):
            relationship_exists = True
            relationship = rel
            break
    return relationship_exists, relationship

# Function to find the shortest path between two tables, if one exists
def shortest_path(rel_list, tab1, tab2):
    rels_to_use_list = []
    # Check that relationship records exist for both tables
    rel_table_set = set()
    for rel in rel_list:
        rel_table_set.add(rel["_from"]["table"])
        rel_table_set.add(rel["to"]["table"])
    if tab1 not in rel_table_set or tab2 not in rel_table_set:
        return rels_to_use_list
    # Check whether direct relationship exists between tables
    direct_exists, direct_rel = confirm_relationship(rel_list, tab1, tab2)
    if direct_exists:
        rels_to_use_list.append(direct_rel)
        return rels_to_use_list
    # Check whether indirect relationship exists between tables
    rel_table_list = list(rel_table_set)
    if len(rel_table_list) > 0:
        set_found = False
        for i in range(3, len(rel_table_list)+1):
            combination_list = list(combinations(rel_table_list, r=i))
            for combo in combination_list:
                rels_to_use_set = set()
                rels_to_use_list = []
                if tab1 in combo and tab2 in combo:
                    path_exists = False
                    for i2 in range(0, i):
                        for i3 in range(0, i):
                            rel_exists, rel = confirm_relationship(rel_list, combo[i2], combo[i3]) 
                            if i2 != i3 and rel_exists:
                                rel_combo = min(combo[i2], combo[i3]) + ":" + max(combo[i2], combo[i3])
                                if rel_combo not in rels_to_use_set:
                                    rels_to_use_set.add(rel_combo)
                                    rels_to_use_list.append(rel)
                                break
                if len(rels_to_use_set) == i-1:
                    set_found = True
                    break
            if set_found == True:
                break
    # Re-order relationships from tab2 to tab1
    rels_to_use_list_ordered = []
    tables_referenced_set = set()
    tables_referenced_set.add(tab2)
    while rels_to_use_list:
        for rel_idx, rel in enumerate(rels_to_use_list):
            if rel["_from"]["table"] in tables_referenced_set and rel["to"]["table"] in tables_referenced_set:
                pass
            elif rel["_from"]["table"] in tables_referenced_set:
                rels_to_use_list_ordered.append(rel)
                tables_referenced_set.add(rel["to"]["table"])
                rels_to_use_list.pop(rel_idx)
            elif rel["to"]["table"] in tables_referenced_set:
                rels_to_use_list_ordered.append(rel)
                tables_referenced_set.add(rel["_from"]["table"])
                rels_to_use_list.pop(rel_idx)
    return rels_to_use_list_ordered

# Function to apply row aggregation logic
def apply_row_agg_logic(stmt, rowagg_obj):
    # Collect partition by parameter - If not present, exit function
    if rowagg_obj["partition_by"] == None or rowagg_obj["partition_by"] == "":
        return stmt
    else:
        partition_parameter = rowagg_obj["partition_by"]
    # Collect order by parameter
    if rowagg_obj["order_by"] == None or rowagg_obj["partition_by"] == "":
        order_parameter = "1"
    else:
        order_parameter = rowagg_obj["order_by"]
    # Apply aggregation function 
    if rowagg_obj["function"] in ("MAX", "MIN"):
        stmt = "{function}({stmt}) OVER (PARTITION BY {part})".format(function= rowagg_obj["function"], stmt=stmt, part=partition_parameter, order=order_parameter)
    elif rowagg_obj["function"] in ("FIRST_VALUE", "LAST_VALUE"):
        stmt = "{function}({stmt} IGNORE NULLS) OVER (PARTITION BY {part} ORDER BY {order})".format(function= rowagg_obj["function"], stmt=stmt, part=partition_parameter, order=order_parameter)
    return stmt

# Function to parse a VOCAB_MAP select statement into its input expressions
def parse_vocab_map_select(stmt):
    tar_expr = re.search("COALESCE\((.*?)\,", stmt).group(1)
    src_expr = re.search("COALESCE\(.*\, (.*)\)", stmt).group(1)
    return src_expr, tar_expr
    
# Function to apply transformation logic
def apply_transformation_logic(stmt, attr_name, transform_obj, made_array, from_clause):
    for transform in transform_obj:
        if transform["function"] == "UUID":
            if made_array == False:
                stmt = "`dsp-data-ingest.transform_resources`.uuid_hash_value({})".format(stmt)
            else:
                stmt = "`dsp-data-ingest.transform_resources`.uuid_hash_array_values({})".format(stmt)
        elif transform["function"] == "HARDCODE":
            hardcode_param = transform["parameters"][0].format(previous=stmt)
            stmt = hardcode_param
        elif transform["function"] == "CUSTOM":
            custom_logic_param = transform["parameters"][0].format(previous=stmt)
            array_param = transform["parameters"][1]
            stmt = custom_logic_param
            if array_param == "array":
                made_array = True
        elif transform["function"] == "VOCAB_MAP":
            attribute_param = transform["parameters"][0]
            from_clause += " LEFT JOIN (SELECT * FROM `dsp-data-ingest.transform_resources.vocab_map` WHERE attribute = '{param}') vm_{param} ON vm_{param}.source_value = {stmt}".format(param=attribute_param, stmt=stmt)
            stmt = "COALESCE(vm_{param}.mapped_value, {stmt})".format(param=attribute_param, stmt=stmt)
        elif transform["function"] == "PREFIX":
            array_param = transform["parameters"][0]
            prefix_param = transform["parameters"][1].format(previous=stmt)
            input_param = transform["parameters"][2].format(previous=stmt)
            if array_param == "array":
                stmt = "`dsp-data-ingest.transform_resources`.prefix_array_values(" + prefix_param + ", " + input_param + ")"
            elif array_param == "non-array":
                stmt = "`dsp-data-ingest.transform_resources`.prefix_value(" + prefix_param + ", " + input_param + ")"
        elif transform["function"] == "VALIDATE_UUID":
            if made_array == False:
                stmt = "`dsp-data-ingest.transform_resources`.validate_uuid_value({})".format(stmt)
            else:
                stmt = "`dsp-data-ingest.transform_resources`.validate_uuid_array_values({})".format(stmt)
        elif transform["function"] == "EXPLODE":
            from_clause += " CROSS JOIN UNNEST({}) {}".format(stmt, attr_name)
            stmt = attr_name
            made_array = False
    return stmt, made_array, from_clause

# Function to construct select statement for a specific field
def construct_select(attr_dict, tar_schema, src_schema, from_clause, where_clause):
    made_array = False
    # Add target field attributes to attribute mapping
    for col in tar_schema["columns"]:
        if attr_dict["name"] == col["name"]:
            attr_dict["datatype"] = col["datatype"]
            attr_dict["array_of"] = col["array_of"]
            if attr_dict["required"] == False:
                if col.get("required") and col["required"] == True:
                    attr_dict["required"] = True
    for pk_col in tar_schema["primaryKey"]:
        if attr_dict["name"] == pk_col:
            attr_dict["required"] = True
    
    # Set base attribute based on number of source fields, target field type, and specified column aggregation
    if len(attr_dict["source"]["fields"]) == 0:
        attr_stmt = ""
    elif len(attr_dict["source"]["fields"]) == 1:
        attr_stmt = attr_dict["source"]["fields"][0]
    else:
        if attr_dict["source"]["with_column_aggregation"] == "FIELD_COALESCE": ## Add support for regular coalesce, but that is complicated with arrays/non-arrays
            attr_stmt = ""
            for field in attr_dict["source"]["fields"]:
                tab_name, col_name = split_field_name(field)
                if confirm_column_exists(src_schema, tab_name, col_name):
                    attr_stmt = field
                    break
        else:
            filtered_field_list = []
            for field in attr_dict["source"]["fields"]:
                tab_name, col_name = split_field_name(field)
                if confirm_column_exists(src_schema, tab_name, col_name):
                    for src_tab in src_schema["tables"]:
                        if src_tab["name"] == tab_name:
                            for src_col in src_tab["columns"]:
                                if src_col["name"] == col_name:
                                    if src_col["array_of"] == True:
                                        array_field = "NULLIF(ARRAY_TO_STRING({}, ','), '')".format(field)
                                        filtered_field_list.append(array_field)
                                    else:
                                        non_array_field = "NULLIF({}, '')".format(field)
                                        filtered_field_list.append(non_array_field)
            field_list_str = ', '.join(filtered_field_list)
            attr_stmt = "SPLIT((SELECT STRING_AGG(DISTINCT col, ',') FROM UNNEST(SPLIT(TRIM(FORMAT('%t', (SELECT AS STRUCT {})), '()'), ', ')) AS col WHERE NOT UPPER(col) = 'NULL'), ',')".format(field_list_str)
            made_array = True
            if attr_dict["array_of"] == False:
                attr_stmt = "ARRAY_TO_STRING({}, ', ')".format(attr_stmt)
                made_array = False
    
    # Apply transformations
    if len(attr_dict["source"]["with_transformation"]) > 0:
        attr_stmt, made_array, from_clause = apply_transformation_logic(attr_stmt, attr_dict["name"], attr_dict["source"]["with_transformation"], made_array, from_clause)
    
    # Apply case logic    
    if attr_dict["source"]["under_condition"] != None:
        attr_stmt = "CASE WHEN " + attr_dict["source"]["under_condition"] + " THEN {} END".format(attr_stmt)
    
    # Apply row aggregation logic
    if attr_dict["source"]["with_row_aggregation"] != None:
        attr_stmt = apply_row_agg_logic(attr_stmt, attr_dict["source"]["with_row_aggregation"])
    
    # Construct final select clauses
    if attr_dict["array_of"] == True and made_array == False:
        attr_stmt = "CASE WHEN {attr} IS NOT NULL THEN [{attr}] ELSE [] END".format(attr=attr_stmt)
    elif attr_dict["array_of"] == False and made_array == True:
        attr_stmt = "ARRAY_TO_STRING({}, ', ')".format(attr_stmt)
    
    # Extend the where_clause for required fields
    if attr_dict["required"] == True:
        if attr_dict["array_of"] == False:
            where_clause += " AND {} IS NOT NULL".format(attr_stmt)
        else:
            where_clause += " AND ARRAY_LENGTH({}) > 0".format(attr_stmt)
    
    # Returned the altered clauses
    return attr_stmt, from_clause, where_clause
    

In [ ]:
## Main Function - Building mapping transformation queries
def build_mapping_query(target_table, src_schema_dict, mapping_spec, bq_project, bq_schema):

    # Initialize function variables
    dataset = bq_project + "." + bq_schema
    query_dict = {"query": None, "syntax_check": None} 

    # Identify the entity of interest and it's place in the mapping specification
    entity_map_idx = None
    entity = target_table["name"]
    for idx, entry in enumerate(mapping_spec["entities"]):
        if entry["name"] == entity:
            entity_map_idx = idx
    if entity_map_idx == None:
        return query_dict

    # Start with mapping_specification, locate the entity of interest, and loop through the record sets associated with it
    final_query_set = set()
    for set_idx, map_set in enumerate(mapping_spec["entities"][entity_map_idx]["record_sets"]):
        
        # Validate whether record set should be processed or skipped
        if not validate_record_set(map_set, src_schema_dict, target_table):
            continue

        # Initialize query variables
        select_clause = "SELECT "
        source_row_agg = ""
        from_clause = ""
        where_clause = "WHERE 1=1"
        
        # Loop through and identify valid attributes in the record set
        attribute_set = set()
        for attr_idx, attr in enumerate(map_set["attributes"]):
            if validate_attribute(attr, src_schema_dict, target_table):
                attribute_set.add(attr["name"])
        
        # Loop through columns in target schema and build attribute, either from record set (if valid) or default logic
        for column_entry in target_table["columns"]:
            if column_entry["name"] in ["source_datarepo_row_ids"]:
                continue
            elif column_entry["name"] not in attribute_set:
                if column_entry["array_of"] == True:
                    select_clause += "[] AS {}, ".format(column_entry["name"])
                else:
                    select_clause += "NULL AS {}, ".format(column_entry["name"])
            else:
                for attr_idx, attr in enumerate(map_set["attributes"]):
                    if attr["name"] == column_entry["name"]:
                        # Build/extend from clause based on source fields
                        for fld_idx, fld in enumerate(attr["source"]["fields"]):
                            tab_name, col_name = split_field_name(fld)
                            if confirm_column_exists(src_schema_dict, tab_name, col_name):
                                if from_clause == "":
                                    base_table, base_field = split_field_name(attr["source"]["fields"][0])
                                    table_iter = 0
                                    table_alias_lookup = {base_table: {"alias": "t" + str(table_iter), "qualified_name": "`" + dataset + "." + base_table + "`"}}
                                    from_clause = "FROM " + base_table
                                    source_row_agg = "ARRAY_AGG(DISTINCT '{}:'||t0.datarepo_row_id)".format(base_table)
                                    source_row_agg = "['{}:'||t{}.datarepo_row_id]".format(base_table, str(table_iter))
                                elif tab_name not in table_alias_lookup:
                                    join_clause = ""
                                    include_join = False
                                    relationships = shortest_path(src_schema_dict["relationships"], tab_name, base_table)
                                    for rel in relationships:
                                        if rel["_from"]["table"] not in table_alias_lookup:
                                            join_clause = " LEFT JOIN " + rel["_from"]["table"] + " ON " + rel["_from"]["table"] + "." + rel["_from"]["column"] + " = " + rel["to"]["table"] + "." + rel["to"]["column"]
                                            from_clause += join_clause
                                            table_iter += 1
                                            table_alias_lookup[rel["_from"]["table"]] = {"alias": "t" + str(table_iter), "qualified_name": "`" + dataset + "." + rel["_from"]["table"] + "`"}
                                            source_row_agg += ", ['{}:'||t{}.datarepo_row_id]".format(rel["_from"]["table"], str(table_iter))
                                        elif rel["to"]["table"] not in table_alias_lookup:
                                            join_clause = " LEFT JOIN " + rel["to"]["table"] + " ON " + rel["to"]["table"] + "." + rel["to"]["column"] + " = " + rel["_from"]["table"] + "." + rel["_from"]["column"]
                                            from_clause += join_clause
                                            table_iter += 1
                                            table_alias_lookup[rel["to"]["table"]] = {"alias": "t" + str(table_iter), "qualified_name": "`" + dataset + "." + rel["to"]["table"] + "`"}
                                            source_row_agg += ", ['{}:'||t{}.datarepo_row_id]".format(rel["to"]["table"], str(table_iter))

                        # Build select clause for attribute 
                        attr_select, from_clause, where_clause = construct_select(attr, target_table, src_schema_dict, from_clause, where_clause)
                        select_clause += "{stmt} AS {name}, ".format(stmt=attr_select, name=attr["name"])
             
        # Build final queries and add to final_query_set
        select_clause += "ARRAY_CONCAT({}) AS source_datarepo_row_ids, ".format(source_row_agg)
        select_clause += "{} AS record_set_priority".format(set_idx)
        initial_root_query = select_clause + " " + from_clause + " " + where_clause
        final_root_query = apply_table_alias(initial_root_query, table_alias_lookup)
        final_query_set.add(final_root_query)

    # Convert final_query_set into a union of queries if multiple record sets are present
    final_entity_query = ""
    if len(final_query_set) == 0:
        return query_dict
    else:
        for idx, entry in enumerate(final_query_set):
            if idx == 0:
                final_entity_query += entry
            else:
                final_entity_query += " UNION ALL " + entry

    # Add deduplication logic 
    pk_col_string = ", ".join(target_table["primaryKey"])
    deduped_entity_query = "SELECT * EXCEPT(record_set_priority, rn) FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY {pk_cols} ORDER BY record_set_priority, 1) AS rn FROM ({query})) WHERE rn = 1".format(pk_cols=pk_col_string, query=final_entity_query)
    query_dict["query"] = deduped_entity_query
    #print(deduped_entity_query)

    # Run syntax check and return query
    smoke_test_query = deduped_entity_query.replace("WHERE 1=1", "WHERE 1=0")
    query_dict["syntax_check"] = run_syntax_check(smoke_test_query)
    return query_dict


In [ ]:
## Secondary function -- Building and executing vocab mapping validation
def evaluate_vocab_mapping(mapping_spec, src_schema_dict, target_schema_dict, bq_project, bq_schema):
    
    # Initialize function variables
    dataset = bq_project + "." + bq_schema
    
    # Determine valid vocab mapping attributes
    vocab_map_valid_attr_set = set()
    for entity in mapping_spec["entities"]:
        target_table = {}
        for table_entry in target_schema_dict["tables"]:
            if table_entry["name"] == entity["name"]:
                target_table = table_entry
        for record_set in entity["record_sets"]:
            if validate_record_set(record_set, src_schema_dict, target_table): 
                for attribute in record_set["attributes"]:
                    for transform in attribute["source"]["with_transformation"]:
                        if transform["function"] == "VOCAB_MAP":
                            if validate_attribute(attribute, src_schema_dict, target_table):
                                vocab_map_valid_attr_set.add(entity["name"] + "." + attribute["name"])
    
    # Loop through valid vocab mapping attributes and build mapping queries
    eval_query_set = set()
    for entity in mapping_spec["entities"]:
        target_table = {}
        for table_entry in target_schema_dict["tables"]:
            if table_entry["name"] == entity["name"]:
                target_table = table_entry
        for record_set in entity["record_sets"]:
            for attr in record_set["attributes"]:
                if (entity["name"] + "." + attr["name"]) in vocab_map_valid_attr_set:
        
                    # Initialize query variables
                    select_clause = "SELECT "
                    from_clause = ""
                    where_clause = "WHERE 1=1"
                    group_by_clause = "GROUP BY "

                    # Build/extend from clause based on source fields
                    for fld_idx, fld in enumerate(attr["source"]["fields"]):
                        tab_name, col_name = split_field_name(fld)
                        if confirm_column_exists(src_schema_dict, tab_name, col_name):
                            if from_clause == "":
                                base_table, base_field = split_field_name(attr["source"]["fields"][0])
                                table_iter = 0
                                table_alias_lookup = {base_table: {"alias": "t" + str(table_iter), "qualified_name": "`" + dataset + "." + base_table + "`"}}
                                from_clause = "FROM " + base_table
                            elif tab_name not in table_alias_lookup:
                                join_clause = ""
                                include_join = False
                                relationships = shortest_path(src_schema_dict["relationships"], tab_name, base_table)
                                for rel in relationships:
                                    if rel["_from"]["table"] not in table_alias_lookup:
                                        join_clause = " LEFT JOIN " + rel["_from"]["table"] + " ON " + rel["_from"]["table"] + "." + rel["_from"]["column"] + " = " + rel["to"]["table"] + "." + rel["to"]["column"]
                                        from_clause += join_clause
                                        table_iter += 1
                                        table_alias_lookup[rel["_from"]["table"]] = {"alias": "t" + str(table_iter), "qualified_name": "`" + dataset + "." + rel["_from"]["table"] + "`"}
                                    elif rel["to"]["table"] not in table_alias_lookup:
                                        join_clause = " LEFT JOIN " + rel["to"]["table"] + " ON " + rel["to"]["table"] + "." + rel["to"]["column"] + " = " + rel["_from"]["table"] + "." + rel["_from"]["column"]
                                        from_clause += join_clause
                                        table_iter += 1
                                        table_alias_lookup[rel["to"]["table"]] = {"alias": "t" + str(table_iter), "qualified_name": "`" + dataset + "." + rel["to"]["table"] + "`"}

                    # Build select clause for attribute 
                    attr_select, from_clause, where_clause = construct_select(attr, target_table, src_schema_dict, from_clause, where_clause)
                    src_expr, map_expr = parse_vocab_map_select(attr_select)
                    select_clause += "'{attr_name}' AS attribute, {src_expr} AS source_value, {map_expr} AS mapped_value, COUNT(*) as record_count".format(attr_name=attr["name"], src_expr=src_expr, map_expr=map_expr)
                    group_by_clause += src_expr + ", " + map_expr
                    
                    # Build final queries and add to final_query_set
                    initial_root_query = select_clause + " " + from_clause + " " + where_clause + " " + group_by_clause
                    final_root_query = apply_table_alias(initial_root_query, table_alias_lookup)
                    eval_query_set.add(final_root_query)
    
    # Convert eval_query_set into a union of queries if vocab map fields are present
    client = bigquery.Client()
    df = pd.DataFrame(columns = ["attribute", "source_value", "mapped_value", "record_count"])
    final_eval_query = ""
    if len(eval_query_set) == 0:
        return df
    else:
        for idx, entry in enumerate(eval_query_set):
            if idx == 0:
                final_eval_query += entry
            else:
                final_eval_query += " UNION ALL " + entry 
        final_eval_query += " ORDER BY attribute, record_count DESC"
    
    # Execute query and return dataframe with results
    try:
        df = df.append(client.query(final_eval_query).result().to_dataframe())
    except Exception as e:
        logging.error("Error during query execution: {}".format(str(e)))
    return df
                             

In [ ]:
# ## TESTING

# # Additional imports
# import data_repo_client
# from google.cloud import storage
# import google.auth
# import google.auth.transport.requests
# import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option("display.max_colwidth", None)
# pd.set_option('display.width', 1000)
# pd.set_option('display.colheader_justify', 'center')
# pd.set_option('display.precision', 3)

# # Setup Google creds and establish TDR clients
# creds, project = google.auth.default()
# auth_req = google.auth.transport.requests.Request()
# creds.refresh(auth_req)
# config = data_repo_client.Configuration()
# config.host = "https://data.terra.bio"
# config.access_token = creds.token
# api_client = data_repo_client.ApiClient(configuration=config)
# api_client.client_side_validation = False
# datasets_api = data_repo_client.DatasetsApi(api_client=api_client)

# # Set the desired mapping specification and retrieve artifacts needed for query construction
# dataset_id = "82bf23c1-fe49-48fe-a19b-cad536f67d41" #params["dataset_id"]
# mapping_target = "anvil"
# mapping_target_spec = "cmg_ext_2" 

# # Retrieve source schema
# src_schema_dict = {}
# try:
#     datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
#     response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
#     src_schema_dict["tables"] = response["schema"]["tables"]
#     src_schema_dict["relationships"] = response["schema"]["relationships"]
#     bq_project = response["access_information"]["big_query"]["project_id"]
#     bq_schema = response["access_information"]["big_query"]["dataset_name"]
# except Exception as e:
#     print("Error retrieving source schema from TDR. Error: {}".format(e))
# #print(src_schema_dict)

# # Retrieve target schema and mapping specification
# target_schema_dict = {}
# mapping_spec = {}
# storage_client = storage.Client()
# bucket = storage_client.get_bucket(ws_bucket_name)
# try:
#     blob = bucket.blob(f"ingest_pipeline/mapping/{mapping_target}/mapping_schema_object.json")
#     target_schema_dict = json.loads(blob.download_as_string(client=None))
# except Exception as e:
#     print("Error retrieving target schema for specified mapping_target. Error: {}".format(e))
# #print(target_schema_dict)
# try:
#     blob = bucket.blob(f"ingest_pipeline/mapping/{mapping_target}/{mapping_target_spec}/mapping_specification.json")
#     blob_string = blob.download_as_text(client=None)
#     blob_string = blob_string.replace("$DATASET_NAME", "Dataset").replace("$PROJECT_NAME", "Project") #UPDATE WITH REAL PARAMETERS
#     mapping_spec = json.loads(blob_string)
# except Exception as e:
#     print("Error retrieving mapping specification for specified mapping_target and mapping_target_spec. Error: {}".format(e))
# #print(mapping_spec)

# # # Evaluate vocab maps
# # df = evaluate_vocab_mapping(mapping_spec, src_schema_dict, target_schema_dict, bq_project, bq_schema)
# # display(df)

# # Build queries from mapping specification
# query_dict = {}
# if target_schema_dict:
#     for target_table in target_schema_dict["tables"]:
#         table_name = target_table["name"]
#         missing_artifacts = False
#         if src_schema_dict and mapping_spec:
#             query_dict[table_name] = build_mapping_query(target_table, src_schema_dict, mapping_spec, bq_project, bq_schema)
#         else:
#             missing_artifacts = True
#             query_dict[table_name] = {"query": "", "syntax_check": ""} 
#     if missing_artifacts == True:
#         print("Source schema dictionary and/or mapping specification missing. Unable to generate queries.")
# else:
#     print("Target schema dictionary missing. Unable to generate queries.")
# query_df = pd.DataFrame.from_dict(query_dict, orient="index")
# query_df.index.name = "target_table"
# query_df.reset_index(inplace=True)
# display(query_df)
